![PPGI_UFRJ](imagens/ppgi-ufrj.png)
# Fundamentos de Ciência de Dados

---
[![DOI](https://zenodo.org/badge/335308405.svg)](https://zenodo.org/badge/latestdoi/335308405)

---
# PPGI/UFRJ 2020.3
## Prof Sergio Serra e Jorge Zavaleta

---
# Módulo 4. Deep Learning

## TensorFlow 2.x

Um **tensor** é um objeto matemático e uma generalização de escalares, vetores e matrizes. Um tensor pode ser representado como uma matriz multidimensional.

Um tensor de ordem zero (rank) é um escalar. Um vetor / matriz é um tensor de ordem 1, enquanto uma matriz é um tensor de ordem 2. Em consequência, um tensor pode ser considerado uma matriz n-dimensional.

Exemplos de tensores:
- 7 : é um tensor de ordem 0. Escalar com shape [].
- [3.,5, 13]: Tensor de ordem 1. É um vetor com shape [3]
- [[10,4,6],[3,4,5]]: É um tensor de ordem 2. É uma matriz com shpe [2,3]
- [[[17,5,1]],[[9,6,3]]]: É um tensor de ordem 3 com shape [2,1,3]

O tensorflow 2.x:

- TensorFlow é uma biblioteca Python de código aberto para computação numérica criada para facilitar o aprendizado de máquina e a resolução de problemas de aprendizado profundo. 
- TensorFlow reúne módulos de aprendizado de máquina, módulos de aprendizado profundo e algoritmos associados em um ambiente de programação comum. 
- TensorFlow 2.x é a versão mais atual do software. 
- Usamos a designação 2.x porque o software está mudando muito rapidamente, atualemente está na versão 2.10.0

Este notebook apresenta conceitos básicos de aprendizagem profunda. O TensorFlow 2.10.0, pode usar o serviço de nuvem do Google, e o Google Drive Interactive para concretizar os conceitos com exemplos de codificação em Python.

### Grafo Computacional e Session v1.0 

O **tensorflow** tem dois programas básico que formam parte do kernel e executam duas ações principais:
- Construir um grafo computacional na fase inicial (fase de construção)
- Rodar ou executar o grafo computacional na fase de execução.

Um **grafo computacional** é uma série de operações do TensorFlow organizadas em nós de um gráfo.

Em **TensorFlow**, se pode configurar um grafo (um grafo padrão). Em seguida, se precisa criar variáveis, marcadores de posição e  valores constantes e, em seguida, se cria a sessão e inicializa as variáveis. Finalmente, se alimenta com os dados os marcadores de posição, de modo a invocar qualquer ação.

Para finalmente avaliar os nós, se deve executar o grafo computacional em uma **sessão**.

### Observação

O TensorFlow 2.x simplificou bastante o desenvolvimento de modelos em comparação com o TensorFlow 1.x. No TensorFlow 2.x, o uso explícito de gráficos computacionais e sessões foi substituído por uma abordagem mais intuitiva e direta:
- *Grafo Computacional*: No TensorFlow 2.x, não é necessário definir gráficos explicitamente. Ele usa a execução ansiosa por padrão, o que significa que as operações são avaliadas imediatamente.
- *Sessão*: As sessões, que eram necessárias para executar o grafo no TensorFlow 1.x, não são mais necessárias no TensorFlow 2.x.

Essa alteração faz com que o TensorFlow 2.x se assemelhe mais ao código Python comum, tornando o desenvolvimento e a depuração muito mais fáceis.

## Keras

**Keras** é uma biblioteca de rede neural de código aberto escrita em Python que pode ser executada em TensorFlow, Microsoft Cognitive Toolkit (CNTK), Theano, R e PlaidML (Keras é uma API) 
- Foi projetada para permitir experimentação rápida em redes neurais profundas.
- Keras é simples e fácil de usar, modular e extensivel, flexível e poderosa. 
- Keras é vista como a biblioteca de aprendizagem profunda preferida pelos iniciantes.

A estrurura de dados básica do Keras são as **camadas** (layer) e os **modelos** (models). Maiores detalhes em [Keras](https://keras.io/)

## Exemplos

Importando as bibliotecas

In [ ]:
# importar as bilbliotecas
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import tensorflow as tf
from tensorflow import keras as ks
#from tensorflow.estimator import LinearRegressor
from keras.models import Sequential 
from keras.layers import Dense
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, r2_score
from scipy import stats
from numpy import sqrt
from sklearn.preprocessing import LabelEncoder
import joblib
#
import warnings
warnings.filterwarnings("ignore")


### Verificar a versão do Tensorflow 

In [ ]:
# 1. importar as bibliotecas
import tensorflow as tf
#
print('tf:',tf.__version__)

### Regressão Linear Usando TensorFlow e Keras

Dataset: Preços das casas em Boston. Dataset descontinuado, a versão utilizada neste exemplo é uma versão modificada de *Harrison, D. and Rubinfeld, D. L. 'Hedonic prices and the demand for clean ar', J. Environ Economics & Management, vol. 5, 81-102, 1978*

Este dataset poder ser utilizado para a previsão da taxa média de ocupação das casas em Boston.


Leitura do dataset

In [ ]:
# caminho do dataset
path = 'data/'
# read dataset
def read_dataset(path,dataset_name):
    ds = pd.read_csv(path+dataset_name,sep=',',encoding='utf-8',low_memory=False,on_bad_lines='skip')#, index_col=0)
    return ds

In [ ]:
# Load data from a CSV file
dataset_name = 'Boston_housing.csv'
houses_data = read_dataset(path,dataset_name)
houses_data.head()

In [ ]:
#shape
houses_data.shape

In [ ]:
# target column (valor médio de las casas) MEDV = PRICE
target_column = houses_data.MEDV
print(target_column)

Análises exploratória de dados

In [ ]:
#EDA
fig, axs = plt.subplots(ncols=7, nrows=2, figsize=(20, 10))
index = 0
axs = axs.flatten()
for k,v in houses_data.items():
    sns.boxplot(y=k, data=houses_data, ax=axs[index])
    index += 1
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)
plt.show()

Colunas como CRIM, ZN, RM e B parecem ter valores discrepantes (*outliers*).

In [ ]:
plt.figure(figsize=(20, 10))
sns.heatmap(houses_data.corr().abs(),annot=True)
plt.show()

Da matriz de correlação, vemos que TAX e RAD são características altamente correlacionadas. As colunas LSTAT, RM, PTRAIO têm uma pontuação de correlação acima de 0,5 com MEDV, o que é uma boa indicação de uso como preditores.

In [ ]:
houses_data['MEDV'] = target_column.astype(np.float32)
houses_data.head()

Analizando as relações entre duas variáveis (distribuições bivariantes)

In [ ]:
# 3. relações entre variaveis
sns.pairplot(houses_data, diag_kind="kde");

Medir o grau de correlação entre duas variáveis. Pode indicar se elas tendem a varaiar juntas.

In [ ]:
# Correlação
correlation_data = houses_data.corr()
correlation_data.style.background_gradient(cmap='coolwarm', axis=None)

Estatística descritiva

In [ ]:
# Estatistica descritiva
stats = houses_data.describe()
houses_stats = stats.transpose()
houses_stats

Selecionar os conjuntos de treinamento (X) e objetivo (Y)

In [ ]:
# Selecionar colunas de dados requeridas (X_data) 
X_data = houses_data[[i for i in houses_data.columns if i not in ['MEDV']]]
# selecionar o conjunto objetivo (Y_data)
Y_data = houses_data[['MEDV']] #MEDV = PRICE

Criar os conjuntos de treinamento e teste

In [ ]:
# Treinamento e teste (80% e 20%)
X_train , X_test ,y_train, y_test = train_test_split(X_data, Y_data, test_size=0.2)

In [ ]:
print('No. de filas no Treinamento: ', X_train.shape[0])
print('No. de filas no Teste: ', X_test.shape[0])
print()
print('No. de colunas no Treinamento: ', X_train.shape[1])
print('No. de colunas no Teste: ', X_test.shape[1])
print()
print('No. de filas nas rótulos de Treinamento: ', y_train.shape[0])
print('No. de filas nos rótulos de Teste: ', y_test.shape[0])
print()
print('No. de colunas nos rótulos de Treinamento: ', y_train.shape[1])
print('No. de colunas nos rótulos de Teste: ', y_test.shape[1])

Normalizar os dados

In [ ]:
# Normalizado os dados
def norm(x):
    stats = x.describe()
    stats = stats.transpose()
    return (x - stats['mean']) / stats['std']

In [ ]:
# normalizar os dados de trenamento
normed_train_features = norm(X_train)
#print(normed_train_features)
# normalizar os dados de teste
normed_test_features = norm(X_test)
#print(normed_test_features)

Pipeline do modelo do tensorflow

In [ ]:
# Contruir o pipeline para modelo do TensorFlow
def feed_input(features_dataframe, target_dataframe,num_of_epochs=10, shuffle=True, batch_size=32):
    #
    def input_feed_function():
        dataset = tf.data.Dataset.from_tensor_slices((dict(features_dataframe), target_dataframe))
        if shuffle:
            dataset = dataset.shuffle(2000)
        dataset = dataset.batch(batch_size).repeat(num_of_epochs)
        return dataset
    #
    return input_feed_function

In [ ]:
# Usando a função definida no pipeline
train_feed_input = feed_input(normed_train_features,y_train)
train_feed_input_testing = feed_input(normed_train_features,y_train, num_of_epochs=1, shuffle=False)
test_feed_input = feed_input(normed_test_features,y_test, num_of_epochs=1, shuffle=False)

Treinamento

In [ ]:
# Modelando o treiamento
feature_columns_numeric = [tf.feature_column.numeric_column(m) for m in X_train.columns]
# modelo
linear_model = tf.estimator.LinearRegressor(feature_columns=feature_columns_numeric, 
                               optimizer='RMSProp',
                               model_dir='D:\\tf')
#
linear_model.train(train_feed_input)

Ajustar o modelo

In [ ]:
# predições
train_predictions = linear_model.predict(train_feed_input_testing)
test_predictions = linear_model.predict(test_feed_input)
train_predictions_series = pd.Series([p['predictions'][0] for p in train_predictions])
test_predictions_series = pd.Series([p['predictions'][0] for p in test_predictions])

Avalia o modelo

In [ ]:
# avalia o modelo
train_predictions_df = pd.DataFrame(train_predictions_series, columns=['predictions'])
test_predictions_df = pd.DataFrame(test_predictions_series, columns=['predictions'])
#
y_train.reset_index(drop=True, inplace=True)
train_predictions_df.reset_index(drop=True, inplace=True)
#
y_test.reset_index(drop=True, inplace=True)
test_predictions_df.reset_index(drop=True, inplace=True)
#
train_labels_with_predictions_df = pd.concat([y_train, train_predictions_df], axis=1)
test_labels_with_predictions_df = pd.concat([y_test,test_predictions_df], axis=1)

Valida o modelo

In [ ]:
# validação
def calculate_errors_and_r2(y_true, y_pred):
    rmse = root_mean_squared_error(y_true, y_pred)
    mse = rmse*2
    r2 = round(r2_score(y_true, y_pred)*100,0)
    return mse, rmse, r2
#
train_mean_squared_error, train_root_mean_squared_error,train_r2_score_percentage = calculate_errors_and_r2(y_train, train_predictions_series)
#
test_mean_squared_error, test_root_mean_squared_error,test_r2_score_percentage = calculate_errors_and_r2(y_test, test_predictions_series)
#
print('Dados de Treinamento - Mean Squared Error = ', train_mean_squared_error)
print('Dados de Treinamento - Root Mean Squared Error = ', train_root_mean_squared_error)
print('Dados de Treinamento - R2 = ', train_r2_score_percentage)
print('Dado de Teste - Mean Squared Error = ', test_mean_squared_error)
print('Dados de Teste - Root Mean Squared Error = ', test_root_mean_squared_error)
print('Dados de Teste - R2 = ', test_r2_score_percentage)

#### Outra forma de fazer o mesmo exemplo

Separar os conjuntos em dados e objetivo (data e target)

In [ ]:
# split into input and output columns
X, y = houses_data.values[:, :-1], houses_data.values[:, -1]


Dividir os conjuntos para treinamento e teste

In [ ]:
# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Determinar o número de características de entrada 

In [ ]:
# determine the number of input features
n_features = X_train.shape[1]

Definir o modelo

In [ ]:
# define model
model = Sequential()
model.add(Dense(10, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
model.add(Dense(8, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1))

Compilar o modelo

In [ ]:
# compile the model
model.compile(optimizer='adam', loss='mse')

Ajustar o modelo

In [ ]:
# fit the model
#model.fit(X_train, y_train, epochs=150, batch_size=32, verbose=0)
history = model.fit(X_train, y_train, epochs=167, batch_size=32, verbose=0, validation_split=0.3)

Avaliar o modelo

In [ ]:
# evaluate the model
#error = model.evaluate(X_test, y_test, verbose=0)
error = model.evaluate(X_test, y_test, verbose=0)
print('MSE: %.3f, RMSE: %.3f' % (error, sqrt(error)))
#print('rmse:%.3f'%rmse)

Fazer previsões com o modelo

In [ ]:
# make a prediction
row = [0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,396.90,4.98]
#yhat = model.predict([row])
yhat = model.predict([row])
print(' Valor previsto de uma casa em Boston: %.3f' % yhat[0][0],'dolares')

Gráfica as curvas de aprendizagem 

In [ ]:
# plot learning curves
plt.title('Curvas de Aprendizagem')
plt.xlabel('Epocas (iterações)')
plt.ylabel('Entropia Cruzada')
plt.plot(history.history['loss'], label='treinamento')
plt.plot(history.history['val_loss'], label='validação')
plt.legend()
plt.show()

## Contsruir uma Rede Neural
Dataset **MNIST dataset** [Dataset](http://yann.lecun.com/exdb/mnist/)

Recohecimento de dígitos manuscritos

In [ ]:
# Load Dataset usando o Keras
(training_images, training_labels), (test_images, test_labels) = ks.datasets.fashion_mnist.load_data()

In [ ]:
# visualizando alguns dados
print('Imagenes de Treinamento - Dataset Shape: {}'.format(training_images.shape))
print('No. de imagens de Treinamento - Dataset Labels: {}'.format(len(training_labels)))
print('Imagens de teste - Dataset Shape: {}'.format(test_images.shape))
print('No. de imagens de Teste - Dataset Labels: {}'.format(len(test_labels)))

Deterinar o tamanho das características de entrada

In [ ]:
# re-escala (0-255) -> 0-1
training_images = training_images / 255.0
test_images = test_images / 255.0

Definir o modelo

In [ ]:
# usando keras para as diferentes camadas
input_data_shape = (28, 28)                                # entrada de 28x28 pixels
#
hidden_activation_function = 'relu'                        # função de ativação - camada oculta
output_activation_function = 'softmax'                     # função de ativação  - camada de saida
#
nn_model = ks.Sequential()                                 # modelo sequencial
#
nn_model.add(ks.layers.Flatten(input_shape=input_data_shape, name='Input_layer'))
nn_model.add(ks.layers.Dense(32, activation=hidden_activation_function, name='Hidden_layer'))
nn_model.add(ks.layers.Dense(10, activation=output_activation_function, name='Output_layer'))
nn_model.summary()

Compilar o modelo

In [ ]:
# otimizando : SGD, RMSprop, Adam, Adadelta, Adagrad, Adamax, Nadam, Ftrl
#
optimizer = 'adam'
loss_function = 'sparse_categorical_crossentropy'
metric = ['accuracy']
#
nn_model.compile(optimizer=optimizer, loss=loss_function, metrics=metric)    # compilar
#
#nn_model.fit(training_images, training_labels, epochs=10)                    # ajustar ao modelo
history = nn_model.fit(training_images, training_labels, epochs=10, batch_size=32, validation_split=0.3) 

Avaliação do treinamento

In [ ]:
# Avaliação do treinamento
training_loss, training_accuracy = nn_model.evaluate(training_images, training_labels)
print('Acurácia dos dados de Treinamento {}'.format(round(float(training_accuracy),2)*100),'%')

Avaliação do teste

In [ ]:
# Avaliação do teste
test_loss, test_accuracy = nn_model.evaluate(test_images,test_labels)
print('Acurácia dos dados de Teste {}'.format(round(float(test_accuracy),2)*100),'%')

Gráficar as curvas de aprendizagem

In [ ]:
# plot learning curves
plt.title('Curvas de aprendizagem')
plt.xlabel('Epocas')
plt.ylabel('Entropia cruzada categórica')
plt.plot(history.history['loss'], label='trainamento')
plt.plot(history.history['val_loss'], label='validação')
plt.legend()
plt.show()

### Dataset: ionosphere.csv

O dataset da ionosfera original do repositório de aprendizado de máquina da UCI é um dataset de classificação binária com dimensionalidade 34. 

Há um atributo com valores todos zeros, que é descartado. Portanto, o número total de dimensões é 33. A classe "ruim" é considerada como classe de outliers e a classe "boa" como inliers.

Leitura do dataset

In [ ]:
# Load data from a CSV file
data_name = 'data/ionosphere.csv'
ionos_data = pd.read_csv(data_name,header=None)
ionos_data.head()

Dividir em conjuntos de  entrada e saída

In [ ]:
# split into input and output columns
X, y = ionos_data.values[:, :-1], ionos_data.values[:, -1]

Conferir os tipos de dados (float32)

In [ ]:
# ensure all data are floating point values
X = X.astype('float32')

Codificar saídas (string) para inteiro

In [ ]:
# encode strings to integer
y = LabelEncoder().fit_transform(y)
#y

Dividir em conjuntos de treinamento e teste

In [ ]:
# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Determinar o número de entradas

In [ ]:
# determine the number of input features
n_features = X_train.shape[1]
n_features

Definir o modelo

In [ ]:
# define o modelo
modelo = Sequential()
modelo.add(Dense(10, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
modelo.add(Dense(8, activation='relu', kernel_initializer='he_normal'))
modelo.add(Dense(1, activation='sigmoid'))

Compilar o modelo

In [ ]:
# compile the model
modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Ajustar o modelo

In [ ]:
# fit the model
y_pred = modelo.fit(X_train, y_train, epochs=150, batch_size=32, verbose=0, validation_split=0.3)

Avaliar o modelo

In [ ]:
# evaluate the model
loss, acc = modelo.evaluate(X_test, y_test, verbose=0)
print('Acurácio do teste: {:.2f}%'.format(100*acc))

Fazer predições

In [ ]:
# make a prediction
valores = [1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300]
yhat = modelo.predict([valores])
print('Valores previstos: {:.1f}%'.format(100*yhat[0][0]))

In [ ]:
# plot learning curves
plt.title('Curvas de aprendizagem ionosphere')
plt.xlabel('Epocas')
plt.ylabel('Entropia cruzada binária')
plt.plot(y_pred.history['loss'], label='trainamento')
plt.plot(y_pred.history['val_loss'], label='validação')
plt.legend()
plt.show()

Resumo do modelo

In [ ]:
# summarize the model
modelo.summary()

Visualizar o gráfico do modelo

In [ ]:
#from tensorflow.keras.utils import plot_model
# summarize the model
tf.keras.utils.plot_model(modelo, 'images/modelo.png', show_shapes=True)
#plot_model(modelo, 'images/modelo.png', show_shapes=True)

Gravar o modelo 'h5'

In [ ]:
# save the model
modelo.save('data/modelo.h5')

Ler o modelo gravado

In [ ]:
# load modelo
modelos = tf.keras.models.load_model('data/modelo.h5')
# make a prediction
val = [1,0,0.99539,-0.05889,0.85243,-0.02306,-0.83398,-0.37708,1,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300]
ym = modelos.predict([val])
print("Previsto: {:.1f}%".format(100*ym[0][0]))

Gravar modelo 'jolib'

In [ ]:
# gravar arquivo jolib
#arquivo_joblib = "data/modelo_treinado_joblib.sav"
#joblib.dump(modelo, arquivo_joblib)

In [ ]:
# Carregando o modelo; Não funciona com keras e tensorflow!!!
#modelo_joblib = joblib.load(arquivo_joblib)
#result = carregando_modelo_joblib.score(previsores_test, classes_test)
#val = [1,0,0.99539,-0.05889,0.85243,-0.02306,-0.83398,-0.37708,1,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300]
#resultado = modelo_joblib.predict([val])
#print(f"Percentual de Acertos {(resultado*100):.1f}%")

Gravando usando Pickle

In [ ]:
# gravar arquivo: funciona na versão python>3.6 e python <3.8
#from pickle5 import pickle
#
#arquivo_pickle = "data/modelo_treinado_pickle.sav"
#pickle.dump(modelo, open(arquivo_pickle, 'wb'))

In [ ]:
# load
# Carregando o modelo
#carregando_modelo_pickle = pickle.load(open(arquivo_pickle, 'rb'))
#
# testando o modelo usando a base de dados de teste
#val = [1,0,0.99539,-0.05889,0.85243,-0.02306,-0.83398,-0.37708,1,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300]
#resultado = modelo_joblib.predict([val])
#result = carregando_modelo_pickle.score(previsores_test, classes_test)
#print(f"Percentual de Acertos {(resultado*100):.2f}%")

---
#### Fundamentos para Ciência Dados &copy; Sergio Serra & Jorge Zavaleta, 2024